# Advanced Classifiers

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from collections import defaultdict
from itertools import cycle
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
# hyperparameter tuning ## To search for the most suitable parameters for the Decision Tree
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from yellowbrick import ROCAUC
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import tree
import pydotplus
from IPython.display import Image

In [ ]:
X_train = pd.read_csv("X_train.txt", header=None,  delim_whitespace=True)
y_train = pd.read_csv("y_train.txt", header=None, delim_whitespace=True )
X_test = pd.read_csv("X_test.txt", header=None, delim_whitespace=True )
y_test = pd.read_csv("y_test.txt", header=None, delim_whitespace=True )
subject_train = pd.read_csv("subject_train.txt", header=None, delim_whitespace=True )
features = pd.read_csv("features.txt", header=None, delim_whitespace=True )

In [ ]:
feature = features
feature.drop(0,inplace=True, axis=1)

In [ ]:
#inseriamo l'intestazione al dataset
lista=[]
feat_transpa = feature.transpose()
for i in range(561):
    lista.append(feat_transpa.iloc[0][i])
X_test.columns=lista    
X_train.columns=lista

In [ ]:
# eliminiamo le colonne che contengono la stima mad in quanto quasi uguale a dev.std
stringa="mad()"
for col in X_train.columns:
    if(stringa in col):
        X_train.drop(labels=col, axis=1, inplace=True)
for col in X_test.columns:
    if(stringa in col):
        X_test.drop(labels=col, axis=1, inplace=True)
#for col in features:
#    if (stringa in col):
#        features.drop(labels=col,axis=1,inplace=True)

## NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB

In [ ]:
clf_nb = GaussianNB()
clf_nb.fit(X_train, np.ravel(y_train))
y_pred_nb = clf_nb.predict(X_test)
y_pred_proba_nb = clf_nb.predict_proba(X_test)

In [ ]:
print('Accuracy %s' % accuracy_score(y_test, y_pred_nb))
print('F1-score %s' % f1_score(y_test, y_pred_nb, average=None))
print(classification_report(y_test, y_pred_nb))

In [ ]:
#Binarize FARE ATTENZIONE PERCHE' DOPO SBALLA LE CLASSI
y_test1 = label_binarize(y_test, classes=[1,2,3,4,5,6])
y_train1 = label_binarize(y_train, classes=[1,2,3,4,5,6])

In [ ]:
# Learn to predict each class against the other
classifier_nb = OneVsRestClassifier(GaussianNB())
classifier_nb.fit(X_train, y_train1)
y_pred_nb_nb = classifier_nb.predict(X_test)
y_pred_proba_nb = classifier_nb.predict_proba(X_test)

In [ ]:
# Compute ROC curve and ROC area for each class
CLASS_LABELS = [ "WALKING" , "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"]

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(CLASS_LABELS)):
    fpr[i], tpr[i], _ = roc_curve(y_test1[:, i], y_pred_proba_nb[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
#plot
lw=2
colors = cycle(["aqua", "darkorange", "cornflowerblue","navy", "deeppink", "gold"])
for i, color in zip(range(len(CLASS_LABELS)), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i+1, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve multiclass Naive Bayes")
plt.legend(loc="lower right")
plt.show()

## Linear Regression 2 dimensions


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
train_A= X_train[['tBodyAcc-std()-X']].values # deicdere quale variabile indipendete
train_B= X_train[['tBodyAcc-max()-Y']].values # decidere quale variabile dipendete
test_A= X_test[['tBodyAcc-std()-X']].values
test_B= X_test[['tBodyAcc-max()-Y']].values

In [ ]:
plt.scatter(train_A, train_B)
plt.xlabel('tBodyAcc-std()-X', fontsize=16)
plt.ylabel('tBodyAcc-max()-Y', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

In [ ]:
reg = LinearRegression()
reg.fit(train_A, train_B)

In [ ]:
print('Coefficients: \n', reg.coef_)
print('Intercept: \n', reg.intercept_)

In [ ]:
y_pred_reg = reg.predict(test_A)

In [ ]:
print('R2: %.3f' % r2_score(test_B, y_pred_reg))
print('MSE: %.3f' % mean_squared_error(test_B, y_pred_reg))
print('MAE: %.3f' % mean_absolute_error(test_B, y_pred_reg))

In [ ]:
fig1, ax1 = plt.subplots()
ax1.scatter(test_A, test_B)
ax1.plot(test_A, y_pred_reg, color='red', linewidth=3)     #Dataframe.T fa il trasposto del df
ax1.set_xlabel('tBodyAcc-std()-X')
ax1.set_ylabel('tBodyAcc-max()-Y')
plt.show()

## Lasso


In [ ]:
from sklearn.linear_model import LassoLarsCV
reg_lasso = LassoLarsCV()
reg_lasso.fit(train_A, train_B)
print('Alpha: \n', reg_lasso.alpha_)
print('Coefficients: \n', reg_lasso.coef_)
print('Intercept: \n', reg_lasso.intercept_)

In [ ]:
y_pred_lasso = reg_lasso.predict(test_A)
print('R2: %.3f' % r2_score(test_B, y_pred_lasso))
print('MSE: %.3f' % mean_squared_error(test_B, y_pred_lasso)) # Misclassification Square Error 
print('MAE: %.3f' % mean_absolute_error(test_B, y_pred_lasso)) #

In [ ]:
fig1, ax1 = plt.subplots()
ax1.scatter(train_A, train_B)
ax1.plot(test_A, y_pred_lasso, color='red', linewidth=3)     #Dataframe.T fa il trasposto del df
ax1.set_xlabel('tBodyAcc-std()-X')
ax1.set_ylabel('tBodyAcc-max()-Y')
plt.show()

## Ridge

In [ ]:
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, ElasticNetCV
reg_rid = RidgeCV()
reg_rid.fit(train_A, train_B)
print('Alpha: \n', reg_lasso.alpha_)
print('Coefficients: \n', reg.coef_)
print('Intercept: \n', reg.intercept_)

In [ ]:
y_pred_rid = reg_rid.predict(test_A)
print('R2: %.3f' % r2_score(test_B, y_pred_rid))
print('MSE: %.3f' % mean_squared_error(test_B, y_pred_rid))
print('MAE: %.3f' % mean_absolute_error(test_B, y_pred_rid))

In [ ]:
fig1, ax1 = plt.subplots()
ax1.scatter(train_A, train_B)
ax1.plot(test_A, y_pred_rid, color='red', linewidth=3)     #Dataframe.T fa il trasposto del df
ax1.set_xlabel('tBodyAcc-std()-X')
ax1.set_ylabel('tBodyAcc-max()-Y')
plt.show()

## Regressione lineare multipla


In [ ]:
attributes = [col for col in X_train.columns if col != 'tBodyAcc-max()-Y']

train_multiple_A= X_train[attributes].values      # tutte le altre variabili indipendenti
test_multiple_A= X_test[attributes].values 

train_multiple_B= X_train[['tBodyAcc-max()-Y']].values # decidere quale variabile dipendete
test_multiple_B= X_test[['tBodyAcc-max()-Y']].values

In [ ]:
reg_multiple = LinearRegression()
reg_multiple.fit(train_multiple_A, train_multiple_B)

In [ ]:
print('Coefficients: \n', reg_multiple.coef_)
print('Intercept: \n', reg_multiple.intercept_)

In [ ]:
y_pred_multiple = reg_multiple.predict(test_multiple_A)

In [ ]:
print('R2: %.6f' % r2_score(test_multiple_B, y_pred_multiple))
print('MSE: %.5f' % mean_squared_error(test_multiple_B, y_pred_multiple))
print('MAE: %.3f' % mean_absolute_error(test_multiple_B, y_pred_multiple))

## Lasso multiple

In [ ]:
# Il Lasso ti butta a zero quelle variabili  che non sono necessarie per spiegare il modello, potresti eliminare la multicollinearità eliminando queste
from sklearn.linear_model import LassoCV

reg_lasso_multiple = LassoCV()
reg_lasso_multiple.fit(train_multiple_A, np.ravel(train_multiple_B))
print('Alpha: \n', reg_lasso_multiple.alpha_)
print('Coefficients: \n', reg_lasso_multiple.coef_)
print('Intercept: \n', reg_lasso_multiple.intercept_)

In [ ]:
y_pred_lasso_multiple = reg_lasso_multiple.predict(test_multiple_A)
print('R2: %.6f' % r2_score(test_multiple_B, y_pred_lasso_multiple))
print('MSE: %.5f' % mean_squared_error(test_multiple_B, y_pred_lasso_multiple))
print('MAE: %.3f' % mean_absolute_error(test_multiple_B, y_pred_lasso_multiple))

## Ridge multiple


In [ ]:
# Come il Lasso penalizza le variabili non utili, ma non le elimina completamente, non le butta proprio a zero ma le mette basse

reg_ridge_multiple = RidgeCV()
reg_ridge_multiple.fit(train_multiple_A, train_multiple_B)
print('Alpha: \n', reg_ridge_multiple.alpha_)
print('Coefficients: \n', reg_ridge_multiple.coef_)
print('Intercept: \n', reg_ridge_multiple.intercept_)

In [ ]:
y_pred_ridge_multiple = reg_ridge_multiple.predict(test_multiple_A)
print('R2: %.6f' % r2_score(test_multiple_B, y_pred_ridge_multiple))
print('MSE: %.5f' % mean_squared_error(test_multiple_B, y_pred_ridge_multiple))
print('MAE: %.3f' % mean_absolute_error(test_multiple_B, y_pred_ridge_multiple))

## Logistic Regression

In [ ]:
# classificazione di tipo binaria, da una risposta del tipo 0 o 1
# crea una classe 0 e 1, la variabile che ci butti dentro deve essere binaria
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

y_train = pd.read_csv("y_train.txt", header=None, delim_whitespace=True )

In [ ]:
y_train_binary=[]
for i in range(0, len(y_train[0])):
    if y_train[0][i]==1:
        y_train_binary.append(1)
    elif y_train[0][i]==2:
        y_train_binary.append(1)
    elif y_train[0][i]==3:
        y_train_binary.append(1)
    else:
        y_train_binary.append(0)

y_test_binary=[]
for i in range(0, len(y_test[0])):
    if y_test[0][i]==1:
        y_test_binary.append(1)
    elif y_test[0][i]==2:
        y_test_binary.append(1)
    elif y_test[0][i]==3:
        y_test_binary.append(1)
    else:
        y_test_binary.append(0)

In [ ]:
selected_columns='tBodyAcc-std()-X'

valori_train=X_train[selected_columns].values
valori_test=X_test[selected_columns].values

#valori_train=X_train.values
#valori_test=X_test.values


In [ ]:
plt.scatter(valori_train, y_train_binary)
plt.xlabel(selected_columns, fontsize=16)
plt.ylabel('Class', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

In [ ]:
clf_log = LogisticRegression(C=1000.0 , random_state=100, solver='liblinear')
# il parametro C è l'inverso della regolarizzazione. E' un fattore di penalizzazione nella missclassification, più grande è + grande sarà il peso della missclassification
#Pertanto, se riduco il valore C, incremento la 
# regolarizzazione. E viceversa.

In [ ]:
clf_log.fit(valori_train.reshape(-1,1), y_train_binary)
y_pred_log = clf_log.predict(valori_test.reshape(-1,1))

print('Accuracy %s' % accuracy_score(y_test_binary, y_pred_log))
print('F1-score %s' % f1_score(y_test_binary, y_pred_log, average=None))
print(classification_report(y_test_binary, y_pred_log))

In [ ]:
from scipy.special import expit

loss_log = expit(sorted(valori_test.reshape(-1,1)) * clf_log.coef_ + clf_log.intercept_).ravel()
plt.plot(sorted(valori_test.reshape(-1,1)), loss_log, color='red', linewidth=3)
plt.scatter(valori_train, np.ravel(y_train_binary))
#plt.xlabel(selected_columns[0], fontsize=16)
#plt.ylabel('Occupancy', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

In [ ]:
reg_reg = LinearRegression()
reg_reg.fit(valori_train.reshape(-1,1), y_train_binary)

In [ ]:
plt.scatter(valori_train.reshape(-1,1), y_train_binary)

loss_log = expit(sorted(valori_test.reshape(-1,1)) * clf_log.coef_ + clf_log.intercept_).ravel()
plt.plot(sorted(valori_test), loss_log, color='orange', linewidth=3)

plt.plot(sorted(valori_test.reshape(-1,1)), reg_reg.coef_ * sorted(valori_test.reshape(-1,1)) + reg_reg.intercept_, color='red', linewidth=3)


#plt.xlabel(selected_columns[0], fontsize=16)
#plt.ylabel('Occupancy', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

In [ ]:
print(confusion_matrix(y_test_binary,y_pred_log,labels=[1,0]))
print("y_test:" ,np.unique(y_test, return_counts=True))
print("y_test_binary:" ,np.unique(y_test_binary, return_counts=True))


## SVC

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
model_svc=SVC()
model_svc.fit(X_train,np.ravel(y_train))
prediction_svc=model_svc.predict(X_test)

In [ ]:
print(classification_report(y_test,prediction_svc))
print(confusion_matrix(y_test,prediction_svc))

### GridSearch for hyperparameter tuning

In [ ]:
param_grid = {'C':[1, 10, 100, 1000], 'kernel' : ['linear']}

In [ ]:
grid= GridSearchCV(SVC(), param_grid, refit=True)

In [ ]:
grid.fit(X_train,np.ravel(y_train))

In [ ]:
grid.best_params_

In [ ]:
predict_svc_grid=grid.predict(X_test)

In [ ]:
print(classification_report(y_test,predict_svc_grid))
print(confusion_matrix(y_test,predict_svc_grid))

In [ ]:
#After GS
clf_SVC = SVC(C=100, gamma=0.01, kernel='rbf')
clf_SVC.fit(X_train, np.ravel(y_train))
clf_SVC = clf_SVC.predict(X_test)

In [ ]:
print(classification_report(y_test, clf_SVC))
print('Accuracy %s' % accuracy_score(y_test, clf_SVC))
print('F1-score %s' % f1_score(y_test, clf_SVC,average=None))